In [1]:
from typing import Final, Dict, Tuple
from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame as SDF
from pyspark.sql.functions import *

In [8]:
spark = SparkSession.builder \
    .master("local[*]")\
    .config("spark.driver.memory", "1500m")\
    .config("spark.memory.offHeap.enabled","true")\
    .config("spark.memory.offHeap.size","1500m") \
    .getOrCreate()

In [ ]:
spark.stop()

In [9]:
#define functions

from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame as SDF
from pyspark.sql.functions import *
def rename_dataframe_cols(df:SDF,col_names: Dict[str,str]) -> SDF:
    """
    Rename dataframe columns
    """
    return df.select(*[col(col_name).alias(col_names.get(col_name,col_name)) for col_name in df.columns])


def update_column_names(df: SDF, index: int) -> SDF:
    df_temp = df
    all_cols = df_temp.columns
    new_cols = dict((column,f"{column}*{index}") for column in all_cols)
    df_temp = df_temp.transform(lambda df_x: rename_dataframe_cols(df_x,new_cols))
    
    return df_temp

In [10]:
def flatten_json(df_arg: SDF, index: int = 1) -> SDF:
    """
    recursive approach to a spark dataframe
    """
    df = update_column_names(df_arg,index) if index == 1 else df_arg
    
    fields = df.schema.fields
    
    for field in fields:
        data_type = str(field.dataType)
        column_name = field.name
        
        first_10_chars = data_type[0:10]
        
        if first_10_chars == 'ArrayType(':
            df_temp = df.withColumn(column_name,explode_outer(col(column_name)))
            return flatten_json(df_temp, index + 1)
        
        elif first_10_chars == 'StructType':
            current_col = column_name
            append_str = current_col
            
            #get data type of column
            
            data_type_str = str(df.schema[current_col].dataType)
            
            #change column name if it exists
            
            df_temp = df.withColumnRenamed(column_name,column_name + "#1") \
                if column_name in data_type_str else df
            current_col = current_col + "#1" if column_name in data_type_str else current_col
            
            #expand struct column values
            df_before_expanding = df_temp.select(f"{current_col}.*")
            newly_gen_cols = df_before_expanding.columns
            
            #find next level go down a level
            begin_index = append_str.rfind('*')
            end_index = len(append_str)
            level = append_str[begin_index +1: end_index]
            next_level = int(level) + 1
            
            #update column names with new level
            
            custom_cols = dict((field,f"{append_str}->{field}*{next_level}") for field in newly_gen_cols)
            df_temp2 = df_temp.select("*",f"{current_col}.*").drop(current_col)
            df_temp3 = df_temp2.transform(lambda df_x: rename_dataframe_cols(df_x,custom_cols))
            return flatten_json(df_temp3,index +1)
            
    return df
    
            
            
            

In [ ]:
json_sample = [
	{
		"id": "0001",
		"type": "donut",
		"name": "Cake",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" },
						{ "id": "1003", "type": "Blueberry" },
						{ "id": "1004", "type": "Devil's Food" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5007", "type": "Powdered Sugar" },
				{ "id": "5006", "type": "Chocolate with Sprinkles" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0002",
		"type": "donut",
		"name": "Raised",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0003",
		"type": "donut",
		"name": "Old Fashioned",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	}
]

In [11]:
file = 'network_rate_sample.json'
# Reading multiline json file
sc = spark.sparkContext
#sc.setSystemProperty("hadoop.home.dir", "C:/Users/feder/Hadoop/")
#RDD = sc.parallelize([json_sample])
df = spark.read.option("multiline","true") \
                      .json(file)


In [13]:
temp_df = df.select("in_network.negotiated_rates")
temp_df.printSchema()

root
 |-- negotiated_rates: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- additional_information: string (nullable = true)
 |    |    |    |    |    |-- billing_class: string (nullable = true)
 |    |    |    |    |    |-- billing_code_modifier: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- expiration_date: string (nullable = true)
 |    |    |    |    |    |-- negotiated_rate: double (nullable = true)
 |    |    |    |    |    |-- negotiated_type: string (nullable = true)
 |    |    |    |    |    |-- service_code: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- provider_references: array (nullable = true)
 

In [ ]:


#df_small = df.limit(100)
#df_small.cache()

In [14]:
df_big = flatten_json(temp_df)

In [ ]:
df.printSchema()

In [15]:
df_big.printSchema()

root
 |-- negotiated_rates*1->provider_references*2: long (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->additional_information*3: string (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->billing_class*3: string (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->billing_code_modifier*3: string (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->expiration_date*3: string (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->negotiated_rate*3: double (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->negotiated_type*3: string (nullable = true)
 |-- negotiated_rates*1->negotiated_prices*2->service_code*3: string (nullable = true)



In [16]:
df_big.toPandas().to_csv("./rates_code.csv", index=False)
#remember to set correct Hadoop environment
#df_big.coalesce(1).write.option("header","true").format("csv").save("./csv_2")
#new_df = df_big.toPandas()